<a href="https://colab.research.google.com/github/teamgaon/KLUE/blob/main/220227_sm_FINAL_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 85.3 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 54.0 MB/s 
     |████████████████████████████████| 134 kB 80.5 MB/s 
     |████████████████████████████████| 1.1 MB 58.4 MB/s 
     |████████████████████████████████| 144 kB 76.4 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 271 kB 84.3 MB/s 
     |████████████████████████████████| 895 kB 71.5 MB/s 
     |████████████████████████████████| 596 kB 82.0 MB/s 
     |████████████████████████████████| 6.8 MB 71.1 MB/s 
     |████████████████████████████████| 1.2 MB 77.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 58 kB 2.9 MB/s 
     |████████████████████████████████| 149.9 

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher

# 데이터

In [5]:
PATH =  '/content/drive/MyDrive/KLUE'

# 데이콘 데이터셋
train = pd.read_csv(os.path.join(PATH, 'train_data.csv'), encoding='utf-8')
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')

# 카카오 데이터셋
kakao_snli = pd.read_csv('/content/drive/MyDrive/KLUE/snli_1.0_train.ko.tsv', sep='\t', encoding='utf-8')
kakao_snli = kakao_snli[:100000]
kakao_dev = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.dev.ko.tsv', sep='\t', encoding='utf-8')
kakao_test = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.test.ko.tsv', sep='\t', encoding='utf-8')
kakao_dev = pd.concat([kakao_dev,kakao_test, kakao_snli])
kakao_dev.rename(columns = {'sentence1':'premise','sentence2':'hypothesis','gold_label':'label'},inplace=True)

# KLUE 데이터셋
klue_dev = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_dev.json')
klue_dev = klue_dev[['premise', 'hypothesis', 'gold_label']]
klue_dev.rename(columns = {'gold_label':'label'}, inplace=True)

# 데이콘, 카카오, KLUE 데이터셋 병합
train = pd.concat([train, klue_dev, kakao_dev], axis=0)
train = train.reset_index(drop=True)
train['index'] = train.index
train = train.dropna()
train = train.reset_index(drop=True)
train['index'] = train.index
train

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
...,...,...,...,...
134514,134514,같은 방향으로 보이는 숲이 우거진 지역에 의상을 입은 사람들이 모여 있다.,사람들은 의상을 입는다,entailment
134515,134515,검은 원피스에 커다란 흰 활을 든 소녀가 카메라를 등지고 서 있다.,하이힐을 신은 남자,contradiction
134516,134516,검은 원피스에 커다란 흰 활을 든 소녀가 카메라를 등지고 서 있다.,서 있는 소녀,entailment
134517,134517,검은 원피스에 커다란 흰 활을 든 소녀가 카메라를 등지고 서 있다.,사진 촬영 준비를 하고 있는 소녀,neutral


# 토크나이저, 함수

In [6]:
from transformers import AutoTokenizer

checkpoint = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [7]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [8]:
def label_to_num(label):
    label_dict = {"entailment": 0, "contradiction": 1, "neutral": 2, "answer": 3}
    num_label = []

    for v in label:
        num_label.append(label_dict[v])
    
    return num_label

In [9]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

In [10]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=16, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=16, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 3
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )

  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220228/model' + str(fold), save_function=accelerator.save)

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['premise']),
  list(train['hypothesis']),
  return_tensors="pt",
  max_length=128, # Max_Length = 190
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['label'])):
  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = label_to_num(train['label'].values)

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------


# 추론

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = label_to_num(test['label'].values)

tokenized_test = tokenizer(
    list(test['premise']),
    list(test['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 3
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220228/model' + str(fold), num_labels=3)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {0: "entailment", 1: "contradiction", 2: "neutral"}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob = pd.DataFrame(output_prob)

  df_label.to_csv('/content/drive/MyDrive/220228/pred_label'+str(fold)+'.csv', index=False)
  df_prob.to_csv('/content/drive/MyDrive/220228/pred_prob'+str(fold)+'.csv', index=False)

## softvoting

In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/220228/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220228/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220228/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220228/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220228/pred_prob4.csv')

In [ ]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)

In [ ]:
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')

In [ ]:
test = pd.concat([test, pred], axis=1)

In [ ]:
# 0: "entailment", 1: "contradiction", 2: "neutral"
test.loc[(test[2] > 0.1) , 'label'] = 'neutral'
test.loc[(test[0] > 0.1) , 'label'] = 'entailment'
test.loc[(test[1] > 0.1) , 'label'] = 'contradiction'

In [ ]:
my_label = []
for index in test[(test[0] > 0.1) & (test[1] > 0.1) & (test[2] > 0.1)].index:
  if test[0].loc[index] > test[1].loc[index] :
    my_label.append('entailment')
  else:
    my_label.append('contradiction')

test.loc[(test[(test[0] > 0.1) & (test[1] > 0.1) & (test[2] > 0.1)].index), 'label'] = my_label

In [ ]:
my_label = []
for index in test[(test[0] > 0.1) & (test[1] > 0.1)].index:
  if test[0].loc[index] > test[1].loc[index] :
    my_label.append('entailment')
  else:
    my_label.append('contradiction')

test.loc[(test[(test[0] > 0.1) & (test[1] > 0.1)].index), 'label'] = my_label

In [ ]:
test.loc[(test[(test['label'] == 'entailment') & (test[2] > 0.7)].index), 'label'] = 'neutral'

In [ ]:
test['label'].value_counts()

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/KLUE/sample_submission.csv')

In [ ]:
submission['label'] = test['label']

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/220228/submission_soft.csv', index=False)

## hardvoting

In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/220228/pred_label0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220228/pred_label1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220228/pred_label2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220228/pred_label3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220228/pred_label4.csv')

In [ ]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']

In [ ]:
df = test
df

In [ ]:
def label_count(df):
  num_neutral = 0
  num_contradiction = 0
  num_entailment = 0
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    if df[col] == 'neutral':
      num_neutral = num_neutral + 1
    if df[col] == 'contradiction':
      num_contradiction = num_contradiction + 1
    if df[col] == 'entailment':
      num_entailment = num_entailment + 1

  return [num_neutral, num_contradiction, num_entailment]

temp = df.apply(label_count, axis=1)
df['temp'] = temp

df['neutral'] = 0
df['contradiction'] = 0
df['entailment'] = 0

def list_to_num(list:list):
  return list[0]
df['neutral'] = df['temp'].map(list_to_num)

def list_to_num(list:list):
  return list[1]
df['contradiction'] = df['temp'].map(list_to_num)

def list_to_num(list:list):
  return list[2]
df['entailment'] = df['temp'].map(list_to_num)

def voting(df):
  cols = ['neutral', 'contradiction', 'entailment']
  for col in cols:
    if df[col] > 2:
      return col
  return 'neutral'

df['label'] = df.apply(voting, axis=1)

In [ ]:
def make_label(list:list):
  if list == [3, 2, 0] :
    return 'contradiction'
  if list == [3, 0, 2]:
    return 'entailment'
  if list == [4, 1, 0]:
    return 'contradiction'
  if list == [4, 0, 1]:
    return 'entailment'
  else :
    return 'answer'

my_label = df['temp'].map(make_label)

In [ ]:
df['my_label'] = my_label

In [ ]:
my_answer = df[df['my_label'] != 'answer']

In [ ]:
df.loc[my_answer.index, 'label'] = my_answer['my_label']

In [ ]:
df['label'].value_counts()

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/KLUE/sample_submission.csv')

In [ ]:
submission['label'] = df['label']

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/220228/submission_hard.csv', index=False)

## soft + hard voting

In [ ]:
soft = pd.read_csv('/content/drive/MyDrive/220228/submission_soft.csv')
hard = pd.read_csv('/content/drive/MyDrive/220228/submission_hard.csv')

In [ ]:
df = test[['premise',	'hypothesis',	'label',	0,	1,	2,	'label0',	'label1',	'label2',	'label3',	'label4']]

In [ ]:
df['soft'] = soft['label']
df['hard'] = hard['label']

In [ ]:
df['soft'].value_counts()

In [ ]:
df['hard'].value_counts()

In [ ]:
df.loc[df[(df['hard'] != df['soft']) & (df['soft'] == 'neutral')].index, 'label'] = df[(df['hard'] != df['soft']) & (df['soft'] == 'neutral')]['soft']

In [ ]:
df.loc[df[(df['hard'] != df['soft']) & (df['soft'] != 'neutral')].index, 'label'] = df[(df['hard'] != df['soft']) & (df['soft'] != 'neutral')]['hard']

In [ ]:
df['label'].value_counts()

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/KLUE/sample_submission.csv')

In [ ]:
submission['label'] = df['label']

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/220228/submission_soft_hard.csv', index=False)